# Downloading Datasets

This is how

In [ ]:
from pyscicat.model import DataFile, OrigDatablock, RawDataset
from scitacean.testing.client import FakeClient
client = FakeClient.from_token(url='fake-url.sci/api/v3',
                               token='fake-token')
pid = '20.500.12269/72fe3ff6-105b-4c7f-b9d0-073b67c90ec3'
client.datasets[pid] = RawDataset(
    pid=pid,
    datasetName='Thaum flux',
    description='Measured the thaum flux',
    createdBy='Ponder Stibbons',
    updatedBy='anonymous',
    updatedAt='2022-11-01T13:22:08.927Z',
    createdAt='2022-08-17T14:20:23.675Z',
    owner='Ponder Stibbons',
    ownerGroup='uu',
    accessGroups=['faculty'],
    principalInvestigator='Ponder Stibbons',
    contactEmail='p.stibbons@uu.am',
    creationTime='2022-06-29T14:01:05.000Z',
    numberOfFiles=2,
    size=641,
    sourceFolder='/hex/ps/thaum')
client.orig_datablocks[pid] = [OrigDatablock(
    id='20.500.12269/02dc390c-811c-4d6a-93bf-9f85a4214ca0',
    datasetId=pid,
    size=641,
    ownerGroup='uu',
    accessGroups=['faculty'],    
    dataFileList=[
        DataFile(path='flux.nxs',
                 size=447,
                 time='2022-08-17T13:54:12Z'),
        DataFile(path='logs/measurement.log',
                 size=194,
                 time='2022-08-17T13:55:21Z'),
    ]
)]

Would normally use

```python
from scitacean import Client
client = Client.from_token(url='https://scicat.ess.eu/api/v3',
                           token='...')
```

In [ ]:
from scitacean import Dataset

In [ ]:
dset = Dataset.from_scicat(client, '20.500.12269/72fe3ff6-105b-4c7f-b9d0-073b67c90ec3')

In [ ]:
dset